In [1]:
import numpy as np
import pandas as pd
import scipy as sp

In [171]:
def binarize(df, column):
    """
    Binarizes a column in a DataFrame by converting a specific value to 1 and all other values to 0.
    
    Parameters:
        df (pandas.DataFrame): The DataFrame containing the column to be binarized.
        column (str): The name of the column to be binarized.
    
    Returns:
        pandas.DataFrame: The DataFrame with the binarized column.
    """
    df[column] = df[column].apply(lambda x: 1 if x == 'spanish' else 0)
    return df
data = pd.read_csv('train.csv')
data = binarize(data, 'label')
data = data.sample(frac=1, random_state=0).reset_index(drop=True)
X = np.array(data['word'])
Y = np.array(data['label'])

In [181]:
import numpy as np
from scipy.stats import chi2_contingency


def extract_features(words):
    # 26 letters + 26^2 bigrams + 26^3 trigrams
    n_features = 26 + 26 ** 2 + 26 ** 3
    features = np.zeros((len(words), n_features))
    
    for i, word in enumerate(words):
        for letter in word:
            index = ord(letter.lower()) - ord('a')
            if 0 <= index < 26:
                features[i, index] += 1
                
        for j in range(len(word) - 1):
            bigram_index = 26 + (ord(word[j].lower()) - ord('a')) * 26 + (ord(word[j+1].lower()) - ord('a'))
            if 0 <= bigram_index - 26 < 26 ** 2:
                features[i, bigram_index] += 1
                
        for k in range(len(word) - 2):
            trigram_index = 26 + 26 ** 2 + (ord(word[k].lower()) - ord('a')) * 26 ** 2 + (ord(word[k+1].lower()) - ord('a')) * 26 + (ord(word[k+2].lower()) - ord('a'))
            if 0 <= trigram_index - 26 - 26 ** 2 < 26 ** 3:
                features[i, trigram_index] += 1
    
    return features





def train_naive_bayes(features, labels):
    # Determine the number of classes and initialize the prior and likelihoods
    classes = np.unique(labels)  # Unique class labels
    n_classes = len(classes)
    n_features = features.shape[1]
    
    # Initializing probabilities
    class_prior = np.zeros(n_classes)
    feature_prob = np.zeros((n_classes, n_features))
    
    for i, cls in enumerate(classes):
        # Find indices where the class label equals the current class
        cls_indices = np.where(labels == cls)[0]
        
        # Calculate class prior probabilities
        class_prior[i] = len(cls_indices) / float(len(labels))
        
        # Subset features for the current class
        cls_features = features[cls_indices, :]
        
        # Calculate the probability of each feature being 1 in the current class
        # Using Laplace smoothing with alpha = 1
        feature_prob[i, :] = (np.sum(cls_features, axis=0) + 1) / (len(cls_indices) + 2)
    
    return class_prior, feature_prob, classes


def predict_naive_bayes(model, features):
    class_prior, feature_likelihood, classes = model
    log_prob = np.log(feature_likelihood)
    # Calculate the log posterior for each class
    log_posterior = np.dot(features, log_prob.T) + np.log(class_prior)
    # Return the class with the highest posterior probability
    return classes[np.argmax(log_posterior, axis=1)]

def chi_square_test(features, labels):
    n_features = features.shape[1]
    chi2_stats = np.zeros(n_features)
    p_values = np.zeros(n_features)
    
    for i in range(n_features):
        # Observed frequencies
        feature_column = features[:, i]
        french_obs = np.sum(feature_column[labels == 0])
        spanish_obs = np.sum(feature_column[labels == 1])
        
        # Expected frequencies based on overall presence and class distribution
        total_french = np.sum(labels == 0)
        total_spanish = np.sum(labels == 1)
        total_presence = french_obs + spanish_obs
        total_words = len(labels)
        
        expected_french = total_presence * (total_french / total_words)
        expected_spanish = total_presence * (total_spanish / total_words)
        
        # Creating contingency table for the feature
        contingency_table = np.array([[french_obs, expected_french],
                                      [spanish_obs, expected_spanish]])
        
        # Applying Chi-squared test
        chi2_stat, p_value, dof, ex = chi2_contingency(contingency_table, correction=False)
        chi2_stats[i] = chi2_stat
        p_values[i] = p_value
    
    return chi2_stats, p_values

def classify(train_words, train_labels, test_words, p_val=0.5):
    train_features = extract_features(train_words)
    test_features = extract_features(test_words)

    model = train_naive_bayes(train_features, train_labels)
    predictions = predict_naive_bayes(model, test_features)
    return predictions


In [185]:
X_train, X_test = X[:900], X[900:]
Y_train, Y_test = Y[:900], Y[900:]

predictions = classify(X_train, Y_train, X_test, p_val=0.5)

In [186]:
# Calculate accuracy
accuracy = np.mean(predictions == Y_test)
print('Accuracy: {:.3f}'.format(accuracy))

Accuracy: 0.743


In [187]:
# find the test words that were misclassified
misclassified = X_test[predictions != Y_test]
# print the misclassified words with their correct and predicted labels
for word, true_label in zip(misclassified, Y_test[predictions != Y_test]):
    print('Word: {}, True label: {} '.format(word, true_label))

Word: tiene, True label: 1 
Word: infinito, True label: 1 
Word: pequena, True label: 1 
Word: repetia, True label: 1 
Word: fascinantes, True label: 1 
Word: charges, True label: 0 
Word: radieux, True label: 0 
Word: parmi, True label: 0 
Word: tandis, True label: 0 
Word: mantuvo, True label: 1 
Word: lente, True label: 1 
Word: neuronales, True label: 1 
Word: fielmente, True label: 1 
Word: perfectamente, True label: 1 
Word: interminables, True label: 1 
Word: siempre, True label: 1 
Word: caracteristiques, True label: 0 
Word: racines, True label: 0 
Word: baker, True label: 0 
Word: aquel, True label: 1 
Word: adoptes, True label: 0 
Word: revelar, True label: 1 
Word: encontro, True label: 1 
Word: representan, True label: 1 
Word: maladie, True label: 0 
Word: parles, True label: 0 
Word: responder, True label: 1 
Word: colores, True label: 1 
Word: quotidien, True label: 0 
Word: chacun, True label: 0 
Word: question, True label: 0 
Word: diferente, True label: 1 
Word: lang

In [109]:


X_preprocessed = extract_features(X_train)
chi2_stats, p_values = chi_square_test(X_preprocessed, Y_train)

c:\Users\limja\anaconda3\envs\CCE\lib\site-packages\scipy\stats\contingency.py:134: RuntimeWarning: invalid value encountered in divide
  expected = reduce(np.multiply, margsums) / observed.sum() ** (d - 1)


In [110]:
selected_features_indices = np.where(p_values < 0.05)[0]